# QA Eval Run
1. Pastikan seluruh dependencies telah terinstall (`requirements.txt`, dan tambah install `ipykernel`)
2. Pastikan folder `chroma_langchain_db` sudah bersisi Collection ChromaDB yang berisi dokumen yang ingin digunakan untuk generasi dataset evaluasi QA
3. Pastikan .env berisi token huggingface dengan nama `HUGGINGFACEHUB_API_TOKEN` 
4. Cek ulang LLM yang dipilih dan konstana lainnya
5. Jalankan seluruh codeblock

In [25]:
import os
import re
import json
from datetime import datetime
from pathlib import Path


import pandas as pd
from tqdm.auto import tqdm

from dotenv import load_dotenv
from datasets import load_dataset
from langchain_huggingface import HuggingFaceEndpoint
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.docstore.document import Document

from extractor import Store, Answerer

tqdm.pandas()  # load tqdm's pandas support
pd.set_option("display.max_colwidth", None)

load_dotenv()

True

In [26]:
JUDGE_MODEL = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
JUDGE_MODEL_CONFIG = {
    "max_new_tokens": 128,
    "top_k": 10,
    "top_p": 0.95,
    "typical_p": 0.95,
    "temperature": 0.01,
    "repetition_penalty": 1.03,
}

DEFENDANT_MODEL_PROVIDER = "hfsi" # hfsi, groq
DEFENDANT_MODEL = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
DEFENDANT_MODEL_CONFIG = {
    "temperature": 0.3,
    "top_p": 0.7,
    "max_tokens": 2048,
}
BASE_DIR = Path("./results")

RATELIMIT_COOLDOWN = 5_000 # ms
EVAL_DATA_CUT = 5 # -1 for no cut

now = datetime.now()
date_time_str = now.strftime("%d.%m.%Y_%H.%M")

In [27]:
vec_store = Store("main")
vec_store.setup()

if DEFENDANT_MODEL_PROVIDER == "hfsi":
    model = DEFENDANT_MODEL

if DEFENDANT_MODEL_PROVIDER == "groq":
    model = ChatGroq(
        model_name=DEFENDANT_MODEL,
        temperature=DEFENDANT_MODEL_CONFIG["temperature"],
        max_tokens=DEFENDANT_MODEL_CONFIG["max_tokens"],
        api_key=os.environ["GROQ_API_KEY"]
    )

defendant = Answerer(vec_store, model=model, **DEFENDANT_MODEL_CONFIG)

### Answer generation and Source Attribution Isolation

In [28]:
with open(BASE_DIR / "eval_data_generated_15.01.25_12.00.json") as f:
    data = json.load(f)
context_question_pairs = pd.DataFrame(data, columns=["context","questions"])[:EVAL_DATA_CUT]

In [29]:
import time

def process_context_questions_pair(x):
    questions, context = x['questions'], [Document(page_content=x['context']),]
    
    answers = []
    answer_citations = []
    for i, q in enumerate(questions):
        # TODO: Add cooldown, bcuz Groq ratelimit shiz
        history, _, citations = defendant.answer_with_search(q, ctx_docs=context, show_cits=False)
        answer_text = history[-1]['content']
        answers.append(answer_text)
        answer_citations.append(citations)
        if i < (len(questions)-1):
            time.sleep(RATELIMIT_COOLDOWN/1000)

    return {"answers": answers, "citations": answer_citations}

In [30]:
llm_answers = context_question_pairs.progress_apply(process_context_questions_pair, axis=1).to_list()
llm_answers = pd.DataFrame(llm_answers)

  0%|          | 0/5 [00:00<?, ?it/s]

In [31]:
context_question_pairs_llm_answers = pd.concat([context_question_pairs, llm_answers], axis=1, ignore_index=False)
context_question_pairs_llm_answers.to_json(BASE_DIR / f"eval_data_questions_context_answers_{date_time_str}.json")

In [32]:
backup_file_datestr = ""
# context_question_pairs_llm_answers = pd.read_json(BASE_DIR / f"eval_data_questions_context_answers_{backup_file_datestr}.json")

In [33]:
IMPROVED_JUDGE_PROMPT = """
You will be given a user_question, system_answer and system_context set.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question depending on the system_context.
Give your answer on a scale of 1 to 4, where 1 means that the system_answer is not helpful at all, and 4 means that the system_answer completely and helpfully addresses the user_question.

Here is the scale you should use to build your answer:
1: The system_answer is terrible: completely irrelevant to the question asked, very partial, or is contradictory to the system_context
2: The system_answer is mostly not helpful: misses some key aspects of the question
3: The system_answer is mostly helpful: provides support, but still could be improved
4: The system_answer is excellent: relevant, direct, detailed, and addresses all the concerns raised in the question

Provide your feedback as follows:

Feedback:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 4)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the context, question, and answer.

Context: {context}
Question: {question}
Answer: {answer}

Feedback:::
Evaluation: """
prompt = PromptTemplate.from_template(IMPROVED_JUDGE_PROMPT)

### LLM-as-a-Judge Code
Diberikan `context`,`question`, dan `answer`, dan di prompt untuk menilai ke-relevanan `answer` dengan `question` dan `context` 

In [34]:
llm = HuggingFaceEndpoint(
    repo_id=JUDGE_MODEL,
    huggingfacehub_api_token=os.environ["HUGGINGFACEHUB_API_TOKEN"],
    **JUDGE_MODEL_CONFIG
)
llm_chain = prompt | llm

In [35]:
def extract_judge_score(judge_responses, split_str: str = "Total rating:") -> int:
    
    judge_responses_scores = []
    for res in judge_responses:
        answer = res
        try:
            if split_str in answer:
                rating = answer.split(split_str)[1]
            else:
                rating = answer
            digit_groups = [el.strip() for el in re.findall(r"\d+(?:\.\d+)?", rating)]
            judge_responses_scores.append(float(digit_groups[0]))
        except Exception as e:
            print(e)
            continue
    
    return {"llm_judge_scores": judge_responses_scores}

In [36]:
def judge_context_questions_answers(x):
    context, question_answer_pairs = x["context"], zip(x["questions"], x["answers"])
    cqa_processed = {
        "llm_judge":[],
        "llm_judge_fixed":[]
    }
    
    for i, qa in enumerate(question_answer_pairs):
        cqa = {
            "context":context,
            "question":qa[0],
            "answer":qa[1],
        }
        result = llm_chain.invoke(cqa)
        
        cqa_processed["llm_judge"].append(result)
        cqa_processed["llm_judge_fixed"].append(result.split("Question")[0])
        if i < (len(x["questions"])-1):
            time.sleep((RATELIMIT_COOLDOWN//2)/1000)
    
    return cqa_processed

In [38]:
cqa_pairs_judged = pd.DataFrame(context_question_pairs_llm_answers.progress_apply(judge_context_questions_answers, axis=1).to_list())

  0%|          | 0/5 [00:00<?, ?it/s]

### Isolasi score dari response

In [39]:
cqa_pairs_judged_scores = pd.DataFrame(cqa_pairs_judged["llm_judge"].apply(extract_judge_score).to_list())

context_question_pairs_llm_answers_judged = pd.concat([context_question_pairs_llm_answers, cqa_pairs_judged, cqa_pairs_judged_scores], axis=1, ignore_index=False)

### Save output

In [40]:
context_question_pairs_llm_answers_judged.to_json(BASE_DIR / f"eval_result_data_{DEFENDANT_MODEL.replace('/','=')}_{date_time_str}.json")

In [42]:
context_question_pairs_llm_answers_judged

context  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    <SENT 01_01>\nAssociations of polymorphisms of eight muscle- or metabolism-related genes\nwith performance in Mount Olympus marathon runners\nGeorgios I. Tsianos,1 Evangelos Evangelou,1 Arnoud Boot,2 M. Carola Zillikens,2 Joyce B. J. van Meurs,2\nAndre G. Uitterlinden,2,3 and John P. A. Ioannidis1,4\n1Department of Hygiene and Epidemiology, University of Ioannina School of Medicine, Ioannina, Greece; Departments\nof 2Internal Medicine and3Epidemiology, Erasmus Medical Center, Rotterdam, The Netherlands; and4Center for Genetic\nEpidemiology and Modeling, Institute for Clinical Research and Health Policy Studies, Tufts Medical Center, Tufts University\nSchool of Medicine, Boston, Massachusetts\nTsianos GI, Evangelou E, Boot A, Zillikens MC, van Meurs JB,\nUitterlinden AG, Ioannidis JP.Associations of polymorphisms of eight\nmuscle- or metabolism-related genes with performance in Mount Olympus marathon runners.J Appl Physiol108: 567–574,2010.\n</SENT 01_01>\n<SENT 01_02>\nFirst published\nDecember 31, 2009; doi:10.1152/japplphysiol.00780.2009.—Athletic endurance performance is probably partly under genetic control, but\ngenetic association studies have yielded inconclusive results.The\nobjective of the present study was to evaluate the association of\npolymorphisms in eight muscle- or metabolism-related genes with\nendurance performance in participants of the Olympus Marathon\nrunning race.We recruited 438 athletes who participated in the 2007\nand 2008 annual running events of the Olympus Marathon: a 43.8-km\nrace with an ascent from sea level to 2,690-m altitude and then a\ndescent to 300 m. Phenotypes of interest were thecompetitive event\ntime at the specific Olympus Marathon where the athlete was enrolled,\nthe fastest reported timing ever achieved in an Olympus Marathon, and\nhow many kilometers per week the athlete ran during the previous year.\n</SENT 01_02>\n<SENT 01_03>\nEleven polymorphisms in/H92513-actinin ( ACTN3), AMP deaminase-1\n(AMPD1), bradykinin B 2 receptor (BDKRB2), /H92522-adrenergic receptor\n(ADRB2), peroxisome proliferator-activated receptor (PPAR)-/H9253coactivator-1/H9251(PPARGC1A), PPAR-/H9251(PPARA), PPAR-/H9254(PPARD), and apoliprotein E (APOE) were evaluated.Hardy-Weinberg equilibrium testing\non the overall cohort of male athletes showed a significant deviation for\nBDKRB2rs1799722 (P /H110050.018;P /H110050.006 when limited to 316 habitual\nmale runners) with an excess of the TT genotype.\n</SENT 01_03>\n<SENT 01_04>\nAcross all athletes